In [1]:
import os
import json
import numpy as np
from multiprocessing import Pool
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Activation, MaxPooling2D, Dropout, Flatten, Dense
import tensorflow as tf


In [2]:
chromaprints_dir = "music_dataset_chromaprints"  # Giữ nguyên đường dẫn cũ
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [3]:
def load_json_file(file_path):
    """
    Load file JSON, trích xuất key "chromaprints" và lấy nhãn từ cấu trúc thư mục.
    Ta giữ nguyên ma trận đặc trưng (không tính trung bình).
    Nếu thành công, trả về tuple (features, label).
    """
    try:
        with open(file_path, "r", encoding="utf-8") as f:
            data = json.load(f)
        chromaprints = data.get("chromaprints", None)
        # Lấy nhãn từ cấu trúc thư mục: folder con đầu tiên của đường dẫn tương đối
        relative_path = os.path.relpath(file_path, chromaprints_dir)
        parts = relative_path.split(os.sep)
        label = parts[0] if parts else None
        if chromaprints is not None and label is not None:
            feat = np.array(chromaprints)
            return (feat, label)
    except Exception as e:
        print(f"Error loading {file_path}: {e}")
    return None

In [ ]:
json_files = []
for root, dirs, files in os.walk(chromaprints_dir):
    for file in files:
        if file.lower().endswith(".json"):
            json_files.append(os.path.join(root, file))
print("Total JSON files found:", len(json_files))

with Pool() as pool:
    results = pool.map(load_json_file, json_files)
results = [r for r in results if r is not None]
features_list = [r[0] for r in results]
labels_list = [r[1] for r in results]
print("Total JSON files loaded:", len(features_list))

Total JSON files found: 214838
